# 🚀 Conexion con la base de datos y librerias **draft_history**

In [1]:
# Importar librerías

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
conexion = sqlite3.connect('c:\\Users\\benav\\Downloads\\nba.sqlite')

In [3]:
# Lee todas las tablas que existen en la base de datos
query = "SELECT name FROM sqlite_master WHERE type='table';"
tablas = pd.read_sql(query, conexion)

print(tablas)

                   name
0                  game
1          game_summary
2           other_stats
3             officials
4      inactive_players
5             game_info
6            line_score
7          play_by_play
8                player
9                  team
10   common_player_info
11         team_details
12         team_history
13  draft_combine_stats
14        draft_history
15     team_info_common


# 🚀 Mirar cuales años son las ultimas temporadas **draft_history**

In [4]:
df_draft = pd.read_sql("SELECT * FROM draft_history", conexion) #tipos de puntos,tiempo etc..

In [5]:
# Obtener los años únicos y ordenarlos ascendente
años_unicos = sorted(df_draft['season'].unique())

# Tomar los últimos 5 años
ultimos_5_años = años_unicos[-5:]

print(ultimos_5_años)


['2019', '2020', '2021', '2022', '2023']


# 🚀 Comenzamos con la primera tabla **other_stats**

In [6]:
#VARIABLE
df_other_stats = pd.read_sql("SELECT * FROM other_stats", conexion) #tipos de puntos,tiempo etc..


miramos cuantos nulos ay en cada columna

In [7]:
print("Valores nulos en df_other_stats:")
print(df_other_stats.isnull().sum())

Valores nulos en df_other_stats:
game_id                      0
league_id                    0
team_id_home                 0
team_abbreviation_home       0
team_city_home               0
pts_paint_home               0
pts_2nd_chance_home          0
pts_fb_home                  0
largest_lead_home            0
lead_changes                 0
times_tied                   0
team_turnovers_home          2
total_turnovers_home       316
team_rebounds_home        1998
pts_off_to_home           2123
team_id_away                 0
team_abbreviation_away       0
team_city_away               0
pts_paint_away               0
pts_2nd_chance_away          0
pts_fb_away                  0
largest_lead_away            0
team_turnovers_away          2
total_turnovers_away       316
team_rebounds_away        1998
pts_off_to_away           2123
dtype: int64


miramos que datos tiene las columnas donde ay nulos para hacerle la limpieza

In [8]:
df_other_stats.dtypes

game_id                    object
league_id                  object
team_id_home               object
team_abbreviation_home     object
team_city_home             object
pts_paint_home              int64
pts_2nd_chance_home         int64
pts_fb_home                 int64
largest_lead_home           int64
lead_changes                int64
times_tied                  int64
team_turnovers_home       float64
total_turnovers_home      float64
team_rebounds_home        float64
pts_off_to_home           float64
team_id_away               object
team_abbreviation_away     object
team_city_away             object
pts_paint_away              int64
pts_2nd_chance_away         int64
pts_fb_away                 int64
largest_lead_away           int64
team_turnovers_away       float64
total_turnovers_away      float64
team_rebounds_away        float64
pts_off_to_away           float64
dtype: object

las columnas con numero y que tiene valores nulos se remplazan por 0 

In [9]:
#se guarda las columnas en una variable
limpieza_other = [
    'team_turnovers_home',
    'team_rebounds_home',
    'pts_off_to_home',
    'total_turnovers_away',
    'team_rebounds_away',
    'pts_off_to_away',
    'total_turnovers_home'
]

#se guarda en una variable la limpieza de NaN
df_other_stats.loc[:, limpieza_other] = df_other_stats.loc[:, limpieza_other].fillna(0)

verificamos los cambios 

In [10]:
print(df_other_stats.isnull().sum())

game_id                   0
league_id                 0
team_id_home              0
team_abbreviation_home    0
team_city_home            0
pts_paint_home            0
pts_2nd_chance_home       0
pts_fb_home               0
largest_lead_home         0
lead_changes              0
times_tied                0
team_turnovers_home       0
total_turnovers_home      0
team_rebounds_home        0
pts_off_to_home           0
team_id_away              0
team_abbreviation_away    0
team_city_away            0
pts_paint_away            0
pts_2nd_chance_away       0
pts_fb_away               0
largest_lead_away         0
team_turnovers_away       2
total_turnovers_away      0
team_rebounds_away        0
pts_off_to_away           0
dtype: int64


# 🚀 Segunda tabla **line_score**

In [11]:
df_line_score = pd.read_sql("SELECT * FROM line_score", conexion) #partidos con puntos por cuarto y extratime

sumamos los nulos para saber que valores tiene las colummnas

In [12]:
print(df_line_score.isnull().sum())

game_date_est                 0
game_sequence             25532
game_id                       0
team_id_home                  0
team_abbreviation_home        0
team_city_name_home           0
team_nickname_home            0
team_wins_losses_home         0
pts_qtr1_home              1004
pts_qtr2_home              1013
pts_qtr3_home              1045
pts_qtr4_home              1044
pts_ot1_home              25759
pts_ot2_home              27051
pts_ot3_home              27243
pts_ot4_home              27270
pts_ot5_home              45577
pts_ot6_home              45578
pts_ot7_home              45578
pts_ot8_home              45578
pts_ot9_home              45578
pts_ot10_home             45578
pts_home                      0
team_id_away                  0
team_abbreviation_away        0
team_city_name_away           0
team_nickname_away            0
team_wins_losses_away         0
pts_qtr1_away              1010
pts_qtr2_away              1013
pts_qtr3_away              1046
pts_qtr4

Filtrar por fecha de los ultimos 5 años 

In [13]:
#  convertir la columna a tipo datetime
df_line_score['game_date_est'] = pd.to_datetime(df_line_score['game_date_est'], errors='coerce')

# filtrar
ultimos_5_años = [2019, 2020, 2021, 2022, 2023]
df_line_score_filtrado = df_line_score[df_line_score['game_date_est'].dt.year.isin(ultimos_5_años)]

#verificar filtrado con unique
print(df_line_score_filtrado['game_date_est'].dt.year.unique())

[2019 2020 2021 2022 2023]


mirar el tipo de datos para remplazar o eliminar

In [14]:
df_line_score_filtrado.dtypes

game_date_est             datetime64[ns]
game_sequence                    float64
game_id                           object
team_id_home                      object
team_abbreviation_home            object
team_city_name_home               object
team_nickname_home                object
team_wins_losses_home             object
pts_qtr1_home                     object
pts_qtr2_home                     object
pts_qtr3_home                     object
pts_qtr4_home                     object
pts_ot1_home                     float64
pts_ot2_home                     float64
pts_ot3_home                     float64
pts_ot4_home                     float64
pts_ot5_home                     float64
pts_ot6_home                     float64
pts_ot7_home                     float64
pts_ot8_home                     float64
pts_ot9_home                     float64
pts_ot10_home                    float64
pts_home                         float64
team_id_away                      object
team_abbreviatio

mirar que valores tiene porque aparece object y no float o int para poder limpiar

In [15]:
for a in ['pts_qtr1_home', 'pts_qtr2_home', 'pts_qtr3_home', 'pts_qtr4_home']:
        print(a)
        print(df_line_score_filtrado[a].unique())

pts_qtr1_home
['27' '24' '25' '28' '41' '32' '26' '35' '29' '21' '23' '30' '19' '22'
 '31' '38' '34' '14' '13' '43' '33' '20' '18' '37' '36' '51' '39' '16'
 '17' '42' '40' '50' '46' '15' '12' '11' '53' '44' '45' '47' '9' '48' '10'
 '49' '55']
pts_qtr2_home
['25' '29' '27' '35' '30' '15' '32' '17' '21' '20' '36' '31' '34' '28'
 '33' '24' '23' '19' '22' '37' '26' '16' '40' '13' '39' '45' '42' '43'
 '48' '18' '38' '44' '41' '14' '12' '11' '47' '46' '51' '10' '9' '49' '50']
pts_qtr3_home
['21' '32' '27' '25' '28' '24' '38' '29' '39' '30' '18' '33' '34' '20'
 '23' '31' '26' '36' '37' '22' '41' '19' '14' '12' '35' '40' '17' '44'
 '45' '13' '15' '42' '11' '8' '16' '43' '46' '48' '49' '9' '10' '47']
pts_qtr4_home
['25' '31' '26' '33' '18' '22' '19' '40' '34' '21' '29' '32' '27' '20'
 '23' '30' '36' '35' '24' '28' '37' '17' '45' '46' '41' '39' '38' '9' '16'
 '15' '11' '43' '42' '14' '48' '8' '44' '13' '6' '12' '10' '0']


comenzamos a rellenar los NaN con 0 ya que son numeros

In [16]:
#se guarda las columnas en una variable
limpieza_score = [
    'game_sequence',
    'pts_qtr1_home',
    'pts_qtr2_home',
    'pts_qtr3_home',
    'pts_qtr4_home',
    'pts_ot1_home',
    'pts_ot2_home',
    'pts_ot3_home',
    'pts_ot4_home',
    'pts_ot5_home',
    'pts_ot6_home',
    'pts_ot7_home',
    'pts_ot8_home',
    'pts_ot9_home',
    'pts_ot10_home',
    'pts_ot9_away',
    'pts_ot10_away'
]

#se guarda en una variable la limpieza de NaN
df_line_score_filtrado.loc[:, limpieza_score] = df_line_score_filtrado.loc[:, limpieza_score].fillna(0)


Verificamos que no haya nulos con la limpieza de antes

In [17]:
print(df_line_score_filtrado.isnull().sum())

game_date_est             0
game_sequence             0
game_id                   0
team_id_home              0
team_abbreviation_home    0
team_city_name_home       0
team_nickname_home        0
team_wins_losses_home     0
pts_qtr1_home             0
pts_qtr2_home             0
pts_qtr3_home             0
pts_qtr4_home             0
pts_ot1_home              0
pts_ot2_home              0
pts_ot3_home              0
pts_ot4_home              0
pts_ot5_home              0
pts_ot6_home              0
pts_ot7_home              0
pts_ot8_home              0
pts_ot9_home              0
pts_ot10_home             0
pts_home                  0
team_id_away              0
team_abbreviation_away    0
team_city_name_away       0
team_nickname_away        0
team_wins_losses_away     0
pts_qtr1_away             0
pts_qtr2_away             0
pts_qtr3_away             0
pts_qtr4_away             0
pts_ot1_away              0
pts_ot2_away              0
pts_ot3_away              0
pts_ot4_away        

# 🚀 Tercera tabla  **common_player_info**

In [18]:
df_common_player_info = pd.read_sql("SELECT * FROM common_player_info", conexion) #similar a player pero mas detallado 

mirar que columnas tiene valores nulos

In [19]:
print(df_common_player_info.isnull().sum())

person_id                             0
first_name                            0
last_name                             0
display_first_last                    0
display_last_comma_first              0
display_fi_last                       0
player_slug                           0
birthdate                             0
school                                1
country                               1
last_affiliation                      0
height                                0
weight                                0
season_exp                            0
jersey                                0
position                              0
rosterstatus                          0
games_played_current_season_flag      0
team_id                               0
team_name                             0
team_abbreviation                     0
team_code                             0
team_city                             0
playercode                            1
from_year                             0


miramos que tipo de dato tiene la columna

In [20]:
print(df_common_player_info[["draft_round","draft_number"]].dtypes)                  

draft_round     object
draft_number    object
dtype: object


verificar si object tiene datos de numero o letras para remplazar

In [21]:
print("draft_number")
print(df_common_player_info["draft_number"].unique())
print("draft round")
print(df_common_player_info["draft_round"].unique())

draft_number
['25' '5' '1' '3' None 'Undrafted' '32' '20' '37' '4' '66' '12' '79' '27'
 '14' '28' '16' '30' '11' '2' '76' '8' '17' '71' '21' '22' '49' '53' '103'
 '39' '35' '92' '23' '146' '73' '36' '54' '10' '26' '18' '15' '51' '31'
 '13' '9' '41' '38' '101' '55' '46' '45' '77' '7' '87' '40' '33' '106'
 '58' '47' '48' '84' '62' '43' '19' '24' '6' '64' '42' '57' '56' '60' '44'
 '147' '34' '139' '81' '72' '29' '52' '69' '50' '165' '109' '91' '70' '95'
 '74' '159' '61' '160' '113' '185' '68' '78' '169' '98' '155' '118' '75'
 '140' '110' '104' '136' '166' '93' '204' '117' '67' '63' '215' '0' '59'
 '65' '86' '85' '100' '90' '82' '122' '158' '115' '88' '149' '80' '127'
 '89' '198' '102' '94' '116' '83' '96' '128' '144' '133' '99' '182' '141'
 '211' '138' '120' '97' '168' '152' '167' '151' '134' '107' '111' '131'
 '145' '162' '150' '221' '202' '190']
draft round
['1' None 'Undrafted' '2' '3' '7' '4' '6' '5' '8' '11' '12' '10' '9' '17'
 '19' '0' '15' '20']


SE REMPLAZA CON Undrafted ya que 0 significa que participo undrafted significa que no fue seleccionado 

In [22]:
df_common_player_info["draft_number"] = df_common_player_info["draft_number"].replace({None: 'Undrafted'}) #undrafted no fue selecionado en el draft
df_common_player_info["draft_round"] = df_common_player_info["draft_round"].replace({None: 'Undrafted'})

verificamos que no hayan mas nulos  con la limpieza anterior 

In [23]:
print(df_common_player_info.isnull().sum())

person_id                           0
first_name                          0
last_name                           0
display_first_last                  0
display_last_comma_first            0
display_fi_last                     0
player_slug                         0
birthdate                           0
school                              1
country                             1
last_affiliation                    0
height                              0
weight                              0
season_exp                          0
jersey                              0
position                            0
rosterstatus                        0
games_played_current_season_flag    0
team_id                             0
team_name                           0
team_abbreviation                   0
team_code                           0
team_city                           0
playercode                          1
from_year                           0
to_year                             0
dleague_flag

# 🚀 Cuarta tabla  **draft_combine_stats**


In [24]:
df_draft_combine_stats = pd.read_sql("SELECT * FROM draft_combine_stats", conexion) #Datos físicos y atléticos

In [25]:
print(df_draft_combine_stats.isnull().sum())

season                             0
player_id                          0
first_name                         0
last_name                          0
player_name                        0
position                           0
height_wo_shoes                   58
height_wo_shoes_ft_in             58
height_w_shoes                   428
height_w_shoes_ft_in             428
weight                            59
wingspan                          58
wingspan_ft_in                    58
standing_reach                    59
standing_reach_ft_in              59
body_fat_pct                     357
hand_length                      794
hand_width                       794
standing_vertical_leap           237
max_vertical_leap                237
lane_agility_time                246
modified_lane_agility_time      1124
three_quarter_sprint             246
bench_press                      538
spot_fifteen_corner_left        1529
spot_fifteen_break_left         1527
spot_fifteen_top_key            1527
s

verificar las 5 ultimas fechas que esten completas

In [26]:
# Obtener todos los valores únicos en orden de aparición
unique_seasons = df_draft_combine_stats["season"].unique()

# Tomar los últimos 5 valores únicos
last_5_unique_seasons = unique_seasons[-5:]

print(last_5_unique_seasons)


['2019' '2020' '2021' '2022' '2023']


trasformar las fechas en datos fecha y hacer el filtro

In [27]:
#cambiar formato a fecha
df_draft_combine_stats["season"] = pd.to_datetime(df_draft_combine_stats["season"], errors='coerce')
#filtrar y guardarlo en una variable 
df_draft_combine_stats_filtrado = df_draft_combine_stats[df_draft_combine_stats['season'].dt.year.isin(ultimos_5_años)]
#verificar que si se filtraron
filtracion = df_draft_combine_stats_filtrado["season"].unique()
filtracion[-5:]

<DatetimeArray>
['2019-01-01 00:00:00', '2020-01-01 00:00:00', '2021-01-01 00:00:00',
 '2022-01-01 00:00:00', '2023-01-01 00:00:00']
Length: 5, dtype: datetime64[ns]

sumar de nuevos los NaN con la fecha filtrada 

In [28]:
print(df_draft_combine_stats_filtrado.isnull().sum())

season                            0
player_id                         0
first_name                        0
last_name                         0
player_name                       0
position                          0
height_wo_shoes                  52
height_wo_shoes_ft_in            52
height_w_shoes                  119
height_w_shoes_ft_in            119
weight                           53
wingspan                         53
wingspan_ft_in                   53
standing_reach                   52
standing_reach_ft_in             52
body_fat_pct                    120
hand_length                      52
hand_width                       52
standing_vertical_leap          103
max_vertical_leap               103
lane_agility_time               103
modified_lane_agility_time      142
three_quarter_sprint            104
bench_press                     307
spot_fifteen_corner_left        352
spot_fifteen_break_left         351
spot_fifteen_top_key            351
spot_fifteen_break_right    

In [29]:
df_draft_combine_stats_filtrado.dtypes

season                          datetime64[ns]
player_id                               object
first_name                              object
last_name                               object
player_name                             object
position                                object
height_wo_shoes                        float64
height_wo_shoes_ft_in                   object
height_w_shoes                         float64
height_w_shoes_ft_in                    object
weight                                  object
wingspan                               float64
wingspan_ft_in                          object
standing_reach                         float64
standing_reach_ft_in                    object
body_fat_pct                            object
hand_length                             object
hand_width                              object
standing_vertical_leap                 float64
max_vertical_leap                      float64
lane_agility_time                      float64
modified_lane

In [30]:
colum_draft = [
    'height_wo_shoes',
    'height_w_shoes',
    'wingspan',
    'standing_reach',
    'standing_vertical_leap',
    'max_vertical_leap',
    'lane_agility_time',
    'modified_lane_agility_time',
    'three_quarter_sprint',
    'bench_press'
]


df_draft_combine_stats_filtrado.loc[:, colum_draft] = df_draft_combine_stats_filtrado[colum_draft].loc[:, colum_draft].fillna(0)


In [31]:
colum_draft_2 = [
    'height_wo_shoes_ft_in',
    'height_w_shoes_ft_in',
    'weight',
    'wingspan_ft_in',
    'standing_reach_ft_in',
    'body_fat_pct',
    'hand_length',
    'hand_width',
    'spot_fifteen_corner_left',
    'off_drib_college_break_right',
    'on_move_fifteen',
    'on_move_college'
]

df_draft_combine_stats_filtrado.loc[:, colum_draft_2] = df_draft_combine_stats_filtrado.loc[:, colum_draft_2].fillna('Sin_Dato')


verificamos que se hayan cambiado lo nulos con la limpieza anterior 

In [32]:
print(df_draft_combine_stats_filtrado.isnull().sum())

season                            0
player_id                         0
first_name                        0
last_name                         0
player_name                       0
position                          0
height_wo_shoes                   0
height_wo_shoes_ft_in             0
height_w_shoes                    0
height_w_shoes_ft_in              0
weight                            0
wingspan                          0
wingspan_ft_in                    0
standing_reach                    0
standing_reach_ft_in              0
body_fat_pct                      0
hand_length                       0
hand_width                        0
standing_vertical_leap            0
max_vertical_leap                 0
lane_agility_time                 0
modified_lane_agility_time        0
three_quarter_sprint              0
bench_press                       0
spot_fifteen_corner_left          0
spot_fifteen_break_left         351
spot_fifteen_top_key            351
spot_fifteen_break_right    

# 🚀 quinta tabla  **player**

In [33]:
df_player = pd.read_csv("c:\\Users\\benav\\Downloads\\player.csv")

In [34]:
df_player.shape

(4831, 5)

In [35]:
print(df_player)

           id            full_name first_name     last_name  is_active
0       76001       Alaa Abdelnaby       Alaa     Abdelnaby          0
1       76002      Zaid Abdul-Aziz       Zaid    Abdul-Aziz          0
2       76003  Kareem Abdul-Jabbar     Kareem  Abdul-Jabbar          0
3          51   Mahmoud Abdul-Rauf    Mahmoud    Abdul-Rauf          0
4        1505    Tariq Abdul-Wahad      Tariq   Abdul-Wahad          0
...       ...                  ...        ...           ...        ...
4826  1627790           Ante Zizic       Ante         Zizic          0
4827    78647             Jim Zoet        Jim          Zoet          0
4828    78648            Bill Zopf       Bill          Zopf          0
4829  1627826          Ivica Zubac      Ivica         Zubac          1
4830    78650           Matt Zunic       Matt         Zunic          0

[4831 rows x 5 columns]


In [36]:
print(df_player.isnull().sum())

id            0
full_name     0
first_name    6
last_name     0
is_active     0
dtype: int64


In [37]:
df_player = df_player.dropna(subset=["first_name"])

In [38]:
print(df_player.isnull().sum())

id            0
full_name     0
first_name    0
last_name     0
is_active     0
dtype: int64


In [39]:
df_player.shape

(4825, 5)

descargar archivos csv de las tablas ya limpias 

In [41]:
df_player.to_csv("CSV/player_limpio.csv", index=False)
df_draft_combine_stats_filtrado.to_csv("CSV/draft_combine_statsr_limpio.csv", index=False)
df_common_player_info.to_csv("CSV/common_player_info.csv", index=False)
df_other_stats.to_csv("CSV/other_stats.csv", index=False)
df_line_score_filtrado.to_csv("CSV/line_score_filtrado.csv", index=False)